# testing model architectures
check models compile okay

In [26]:
import torch
import numpy as np
from torchinfo import summary
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
class Kiunet(nn.Module):
    
    def __init__(self):
        super().__init__()
        

        self.encoder1 = nn.Conv2d(1, 16, 3, stride=1, padding=1)  # First Layer GrayScale Image , change to input channels to 3 in case of RGB 
        self.en1_bn = nn.BatchNorm2d(16)
        self.encoder2=   nn.Conv2d(16, 32, 3, stride=1, padding=1)  
        self.en2_bn = nn.BatchNorm2d(32)
        self.encoder3=   nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.en3_bn = nn.BatchNorm2d(64)

        self.decoder1 =   nn.Conv2d(64, 32, 3, stride=1, padding=1)   
        self.de1_bn = nn.BatchNorm2d(32)
        self.decoder2 =   nn.Conv2d(32,16, 3, stride=1, padding=1)
        self.de2_bn = nn.BatchNorm2d(16)
        self.decoder3 =   nn.Conv2d(16, 8, 3, stride=1, padding=1)
        self.de3_bn = nn.BatchNorm2d(8)

        self.decoderf1 =   nn.Conv2d(64, 32, 3, stride=1, padding=1)
        self.def1_bn = nn.BatchNorm2d(32)
        self.decoderf2=   nn.Conv2d(32, 16, 3, stride=1, padding=1)
        self.def2_bn = nn.BatchNorm2d(16)
        self.decoderf3 =   nn.Conv2d(16, 8, 3, stride=1, padding=1)
        self.def3_bn = nn.BatchNorm2d(8)

        self.encoderf1 =   nn.Conv2d(1, 16, 3, stride=1, padding=1)  # First Layer GrayScale Image , change to input channels to 3 in case of RGB 
        self.enf1_bn = nn.BatchNorm2d(16)
        self.encoderf2=   nn.Conv2d(16, 32, 3, stride=1, padding=1)
        self.enf2_bn = nn.BatchNorm2d(32)
        self.encoderf3 =   nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.enf3_bn = nn.BatchNorm2d(64)

        self.intere1_1 = nn.Conv2d(16,16,3, stride=1, padding=1)
        self.inte1_1bn = nn.BatchNorm2d(16)
        self.intere2_1 = nn.Conv2d(32,32,3, stride=1, padding=1)
        self.inte2_1bn = nn.BatchNorm2d(32)
        self.intere3_1 = nn.Conv2d(64,64,3, stride=1, padding=1)
        self.inte3_1bn = nn.BatchNorm2d(64)

        self.intere1_2 = nn.Conv2d(16,16,3, stride=1, padding=1)
        self.inte1_2bn = nn.BatchNorm2d(16)
        self.intere2_2 = nn.Conv2d(32,32,3, stride=1, padding=1)
        self.inte2_2bn = nn.BatchNorm2d(32)
        self.intere3_2 = nn.Conv2d(64,64,3, stride=1, padding=1)
        self.inte3_2bn = nn.BatchNorm2d(64)

        self.interd1_1 = nn.Conv2d(32,32,3, stride=1, padding=1)
        self.intd1_1bn = nn.BatchNorm2d(32)
        self.interd2_1 = nn.Conv2d(16,16,3, stride=1, padding=1)
        self.intd2_1bn = nn.BatchNorm2d(16)
        self.interd3_1 = nn.Conv2d(64,64,3, stride=1, padding=1)
        self.intd3_1bn = nn.BatchNorm2d(64)

        self.interd1_2 = nn.Conv2d(32,32,3, stride=1, padding=1)
        self.intd1_2bn = nn.BatchNorm2d(32)
        self.interd2_2 = nn.Conv2d(16,16,3, stride=1, padding=1)
        self.intd2_2bn = nn.BatchNorm2d(16)
        self.interd3_2 = nn.Conv2d(64,64,3, stride=1, padding=1)
        self.intd3_2bn = nn.BatchNorm2d(64)

        self.final = nn.Conv2d(8,2,1,stride=1,padding=0)
        
        self.soft = nn.Softmax(dim =1)
    
    def forward(self, x):

        out = F.relu(self.en1_bn(F.max_pool2d(self.encoder1(x),2,2)))  #U-Net branch
        out1 = F.relu(self.enf1_bn(F.interpolate(self.encoderf1(x),scale_factor=(2,2),mode ='bilinear'))) #Ki-Net branch
        
        print("unet out shape:  ", out.shape)
        print("kinet out shape: ", out1.shape)
        print("------------------\n")
        
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.inte1_1bn(self.intere1_1(out1))),scale_factor=(0.25,0.25),mode ='bilinear')) #CRFB
        out1 = torch.add(out1,F.interpolate(F.relu(self.inte1_2bn(self.intere1_2(tmp))),scale_factor=(4,4),mode ='bilinear')) #CRFB
        print("post crfb unet out shape:  ", out.shape)
        print("post crfb kinet out shape: ", out1.shape)
        print("------------------\n")
        u1 = out  #skip conn
        o1 = out1  #skip conn

        out = F.relu(self.en2_bn(F.max_pool2d(self.encoder2(out),2,2)))
        out1 = F.relu(self.enf2_bn(F.interpolate(self.encoderf2(out1),scale_factor=(2,2),mode ='bilinear')))
        
        print("unet out shape:  ", out.shape)
        print("kinet out shape: ", out1.shape)
        print("------------------\n")
        
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.inte2_1bn(self.intere2_1(out1))),scale_factor=(0.0625,0.0625),mode ='bilinear'))
        out1 = torch.add(out1,F.interpolate(F.relu(self.inte2_2bn(self.intere2_2(tmp))),scale_factor=(16,16),mode ='bilinear'))
        print("post crfb unet out shape:  ", out.shape)
        print("post crfb kinet out shape: ", out1.shape)
        print("------------------\n")
        u2 = out
        o2 = out1

        out = F.relu(self.en3_bn(F.max_pool2d(self.encoder3(out),2,2)))
        out1 = F.relu(self.enf3_bn(F.interpolate(self.encoderf3(out1),scale_factor=(2,2),mode ='bilinear')))
        
        print("unet out shape:  ", out.shape)
        print("kinet out shape: ", out1.shape)
        print("------------------\n")
        
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.inte3_1bn(self.intere3_1(out1))),scale_factor=(0.015625,0.015625),mode ='bilinear'))
        out1 = torch.add(out1,F.interpolate(F.relu(self.inte3_2bn(self.intere3_2(tmp))),scale_factor=(64,64),mode ='bilinear'))
        print("post crfb unet out shape:  ", out.shape)
        print("post crfb kinet out shape: ", out1.shape)
        print("------------------\n")
        ### End of encoder block

        print("STARTING decoder\n")
        ### Start Decoder
        print("unet in hape", out.shape)
        print("kinet in shape", out1.shape)
        print()
        
        out = F.relu(self.de1_bn(F.interpolate(self.decoder1(out),scale_factor=(2,2),mode ='bilinear')))  #U-NET
        out1 = F.relu(self.def1_bn(F.max_pool2d(self.decoderf1(out1),2,2))) #Ki-NET
        print("unet out shape:  ", out.shape)
        print("kinet out shape: ", out1.shape)
        print("------------------\n")
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.intd1_1bn(self.interd1_1(out1))),scale_factor=(0.0625,0.0625),mode ='bilinear'))
        out1 = torch.add(out1,F.interpolate(F.relu(self.intd1_2bn(self.interd1_2(tmp))),scale_factor=(16,16),mode ='bilinear'))
        print("\nunet shape crfb 1", out.shape)
        print("kinet shape crfb 1", out1.shape)
        
        out = torch.add(out,u2)  #skip conn
        out1 = torch.add(out1,o2)  #skip conn
        
        print("unet skip out shape:  ", out.shape)
        print("kinet skip out shape: ", out1.shape)
        print("------------------\n")
        

        out = F.relu(self.de2_bn(F.interpolate(self.decoder2(out),scale_factor=(2,2),mode ='bilinear')))
        out1 = F.relu(self.def2_bn(F.max_pool2d(self.decoderf2(out1),2,2)))
        
        print("unet out shape:  ", out.shape)
        print("kinet out shape: ", out1.shape)
        print("------------------\n")
        
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.intd2_1bn(self.interd2_1(out1))),scale_factor=(0.25,0.25),mode ='bilinear'))
        out1 = torch.add(out1,F.interpolate(F.relu(self.intd2_2bn(self.interd2_2(tmp))),scale_factor=(4,4),mode ='bilinear'))
        print("post crfb unet out shape:  ", out.shape)
        print("post crfb kinet out shape: ", out1.shape)
        print("------------------\n")
        
        out = torch.add(out,u1)
        out1 = torch.add(out1,o1)
        
        print("unet skip out shape:  ", out.shape)
        print("kinet skip out shape: ", out1.shape)
        print("------------------\n")

        out = F.relu(self.de3_bn(F.interpolate(self.decoder3(out),scale_factor=(2,2),mode ='bilinear')))
        out1 = F.relu(self.def3_bn(F.max_pool2d(self.decoderf3(out1),2,2)))

        print("unet out shape:  ", out.shape)
        print("kinet out shape: ", out1.shape)
        print("------------------\n")

        out = torch.add(out,out1) # fusion of both branches

        out = F.relu(self.final(out))  #1*1 conv
        

        # out = self.soft(out)
        
        return out


In [12]:
kiu = Kiunet()

In [13]:
summary(kiu, (1, 1, 64, 64))

unet out shape:   torch.Size([1, 16, 32, 32])
kinet out shape:  torch.Size([1, 16, 128, 128])
------------------

post crfb unet out shape:   torch.Size([1, 16, 32, 32])
post crfb kinet out shape:  torch.Size([1, 16, 128, 128])
------------------

unet out shape:   torch.Size([1, 32, 16, 16])
kinet out shape:  torch.Size([1, 32, 256, 256])
------------------

post crfb unet out shape:   torch.Size([1, 32, 16, 16])
post crfb kinet out shape:  torch.Size([1, 32, 256, 256])
------------------

unet out shape:   torch.Size([1, 64, 8, 8])
kinet out shape:  torch.Size([1, 64, 512, 512])
------------------

post crfb unet out shape:   torch.Size([1, 64, 8, 8])
post crfb kinet out shape:  torch.Size([1, 64, 512, 512])
------------------

STARTING decoder

unet in hape torch.Size([1, 64, 8, 8])
kinet in shape torch.Size([1, 64, 512, 512])

unet out shape:   torch.Size([1, 32, 16, 16])
kinet out shape:  torch.Size([1, 32, 256, 256])
------------------


unet shape crfb 1 torch.Size([1, 32, 16, 1

Layer (type:depth-idx)                   Output Shape              Param #
Kiunet                                   --                        --
├─Conv2d: 1-1                            [1, 16, 64, 64]           160
├─BatchNorm2d: 1-2                       [1, 16, 32, 32]           32
├─Conv2d: 1-3                            [1, 16, 64, 64]           160
├─BatchNorm2d: 1-4                       [1, 16, 128, 128]         32
├─Conv2d: 1-5                            [1, 16, 128, 128]         2,320
├─BatchNorm2d: 1-6                       [1, 16, 128, 128]         32
├─Conv2d: 1-7                            [1, 16, 32, 32]           2,320
├─BatchNorm2d: 1-8                       [1, 16, 32, 32]           32
├─Conv2d: 1-9                            [1, 32, 32, 32]           4,640
├─BatchNorm2d: 1-10                      [1, 32, 16, 16]           64
├─Conv2d: 1-11                           [1, 32, 128, 128]         4,640
├─BatchNorm2d: 1-12                      [1, 32, 256, 256]         64
├

In [19]:
x = torch.rand((1,1,64,64))

In [22]:
x.shape

torch.Size([1, 1, 64, 64])

In [23]:
with torch.no_grad():
    y = kiu(x.to("cuda")).cpu()

unet out shape:   torch.Size([1, 16, 32, 32])
kinet out shape:  torch.Size([1, 16, 128, 128])
------------------

post crfb unet out shape:   torch.Size([1, 16, 32, 32])
post crfb kinet out shape:  torch.Size([1, 16, 128, 128])
------------------

unet out shape:   torch.Size([1, 32, 16, 16])
kinet out shape:  torch.Size([1, 32, 256, 256])
------------------

post crfb unet out shape:   torch.Size([1, 32, 16, 16])
post crfb kinet out shape:  torch.Size([1, 32, 256, 256])
------------------

unet out shape:   torch.Size([1, 64, 8, 8])
kinet out shape:  torch.Size([1, 64, 512, 512])
------------------

post crfb unet out shape:   torch.Size([1, 64, 8, 8])
post crfb kinet out shape:  torch.Size([1, 64, 512, 512])
------------------

STARTING decoder

unet in hape torch.Size([1, 64, 8, 8])
kinet in shape torch.Size([1, 64, 512, 512])

unet out shape:   torch.Size([1, 32, 16, 16])
kinet out shape:  torch.Size([1, 32, 256, 256])
------------------


unet shape crfb 1 torch.Size([1, 32, 16, 1

In [24]:
y.shape

torch.Size([1, 2, 64, 64])

In [25]:
class kiunet3d(nn.Module): #

    def __init__(self, c=4,n=1,channels=128,groups = 16,norm='bn', num_classes=5):
        super(kiunet3d, self).__init__()

        # Entry flow
        self.encoder1 = nn.Conv3d( c, n, kernel_size=3, padding=1, stride=1, bias=False)# H//2
        self.encoder2 = nn.Conv3d( n, 2*n, kernel_size=3, padding=1, stride=1, bias=False)
        self.encoder3 = nn.Conv3d( 2*n, 4*n, kernel_size=3, padding=1, stride=1, bias=False)
        
        self.kencoder1 = nn.Conv3d( c, n, kernel_size=3, padding=1, stride=1, bias=False)
        self.kencoder2 = nn.Conv3d( n, 2*n, kernel_size=3, padding=1, stride=1, bias=False)
        self.kencoder3 = nn.Conv3d( 2*n, 2*n, kernel_size=3, padding=1, stride=1, bias=False)

        self.downsample1 = nn.MaxPool3d(2, stride=2)
        self.downsample2 = nn.MaxPool3d(2, stride=2)
        self.downsample3 = nn.MaxPool3d(2, stride=2)
        self.kdownsample1 = nn.MaxPool3d(2, stride=2)
        self.kdownsample2 = nn.MaxPool3d(2, stride=2)
        self.kdownsample3 = nn.MaxPool3d(2, stride=2)
        

        self.upsample1 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False) # H//8        
        self.upsample2 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False) # H//4
        self.upsample3 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False) # H//2
        self.kupsample1 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False) # H//8        
        self.kupsample2 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False) # H//4
        self.kupsample3 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False) # H//2
        
        self.decoder1 = nn.Conv3d( 4*n, 2*n, kernel_size=3, padding=1, stride=1, bias=False)        
        self.decoder2 = nn.Conv3d( 2*n, 2*n, kernel_size=3, padding=1, stride=1, bias=False)        
        self.decoder3 = nn.Conv3d( 2*n, c, kernel_size=3, padding=1, stride=1, bias=False)        
        self.kdecoder1 = nn.Conv3d( 2*n, 2*n, kernel_size=3, padding=1, stride=1, bias=False)        
        self.kdecoder2 = nn.Conv3d( 2*n, 2*n, kernel_size=3, padding=1, stride=1, bias=False)        
        self.kdecoder3 = nn.Conv3d( 2*n, c, kernel_size=3, padding=1, stride=1, bias=False)        
        
        self.intere1_1 = nn.Conv3d(n,n,3, stride=1, padding=1)
        # self.inte1_1bn = nn.BatchNorm2d(16)
        self.intere2_1 = nn.Conv3d(2*n,2*n,3, stride=1, padding=1)
        # self.inte2_1bn = nn.BatchNorm2d(32)
        self.intere3_1 = nn.Conv3d(2*n,4*n,3, stride=1, padding=1)
        # self.inte3_1bn = nn.BatchNorm2d(64)

        self.intere1_2 = nn.Conv3d(n,n,3, stride=1, padding=1)
        # self.inte1_2bn = nn.BatchNorm2d(16)
        self.intere2_2 = nn.Conv3d(2*n,2*n,3, stride=1, padding=1)
        # self.inte2_2bn = nn.BatchNorm2d(32)
        self.intere3_2 = nn.Conv3d(4*n,2*n,3, stride=1, padding=1)
        # self.inte3_2bn = nn.BatchNorm2d(64)

        self.interd1_1 = nn.Conv3d(2*n,2*n,3, stride=1, padding=1)
        # self.intd1_1bn = nn.BatchNorm2d(32)
        self.interd2_1 = nn.Conv3d(2*n,2*n,3, stride=1, padding=1)
        # self.intd2_1bn = nn.BatchNorm2d(16)
        self.interd3_1 = nn.Conv3d(n,n,3, stride=1, padding=1)
        # self.intd3_1bn = nn.BatchNorm2d(64)

        self.interd1_2 = nn.Conv3d(2*n,2*n,3, stride=1, padding=1)
        # self.intd1_2bn = nn.BatchNorm2d(32)
        self.interd2_2 = nn.Conv3d(2*n,2*n,3, stride=1, padding=1)
        # self.intd2_2bn = nn.BatchNorm2d(16)
        self.interd3_2 = nn.Conv3d(n,n,3, stride=1, padding=1)
        # self.intd3_2bn = nn.BatchNorm2d(64)

        self.seg = nn.Conv3d(c, num_classes, kernel_size=1, padding=0,stride=1,bias=False)

        self.softmax = nn.Softmax(dim=1)

        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                torch.nn.init.torch.nn.init.kaiming_normal_(m.weight) #
            elif isinstance(m, nn.BatchNorm3d) or isinstance(m, nn.GroupNorm):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Encoder
        out = F.relu(F.max_pool3d(self.encoder1(x),2,2))  #U-Net branch
        out1 = F.relu(F.interpolate(self.kencoder1(x),scale_factor=2,mode ='trilinear')) #Ki-Net branch
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.intere1_1(out1)),scale_factor=0.25,mode ='trilinear')) #CRFB
        out1 = torch.add(out1,F.interpolate(F.relu(self.intere1_2(tmp)),scale_factor=4,mode ='trilinear')) #CRFB
        
        u1 = out  #skip conn
        o1 = out1  #skip conn

        out = F.relu(F.max_pool3d(self.encoder2(out),2,2))
        out1 = F.relu(F.interpolate(self.kencoder2(out1),scale_factor=2,mode ='trilinear'))
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.intere2_1(out1)),scale_factor=0.0625,mode ='trilinear'))
        out1 = torch.add(out1,F.interpolate(F.relu(self.intere2_2(tmp)),scale_factor=16,mode ='trilinear'))
        
        u2 = out
        o2 = out1

        out = F.relu(F.max_pool3d(self.encoder3(out),2,2))
        out1 = F.relu(F.interpolate(self.kencoder3(out1),scale_factor=2,mode ='trilinear'))
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.intere3_1(out1)),scale_factor=0.015625,mode ='trilinear'))
        out1 = torch.add(out1,F.interpolate(F.relu(self.intere3_2(tmp)),scale_factor=64,mode ='trilinear'))
        
        ### End of encoder block

        ### Start Decoder
        
        out = F.relu(F.interpolate(self.decoder1(out),scale_factor=2,mode ='trilinear'))  #U-NET
        out1 = F.relu(F.max_pool3d(self.kdecoder1(out1),2,2)) #Ki-NET
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.interd1_1(out1)),scale_factor=0.0625,mode ='trilinear'))
        out1 = torch.add(out1,F.interpolate(F.relu(self.interd1_2(tmp)),scale_factor=16,mode ='trilinear'))
        
        out = torch.add(out,u2)  #skip conn
        out1 = torch.add(out1,o2)  #skip conn

        out = F.relu(F.interpolate(self.decoder2(out),scale_factor=2,mode ='trilinear'))
        out1 = F.relu(F.max_pool3d(self.kdecoder2(out1),2,2))
        tmp = out
        out = torch.add(out,F.interpolate(F.relu(self.interd2_1(out1)),scale_factor=0.25,mode ='trilinear'))
        out1 = torch.add(out1,F.interpolate(F.relu(self.interd2_2(tmp)),scale_factor=4,mode ='trilinear'))
        
        out = torch.add(out,u1)
        out1 = torch.add(out1,o1)

        out = F.relu(F.interpolate(self.decoder3(out),scale_factor=2,mode ='trilinear'))
        out1 = F.relu(F.max_pool3d(self.kdecoder3(out1),2,2))

        

        out = torch.add(out,out1) # fusion of both branches

        out = F.relu(self.seg(out))  #1*1 conv
        

        # out = self.soft(out)
        return out

In [27]:
net3d = kiunet3d()

In [30]:
summary(net3d, (1, 2, 25, 64, 64))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []